# Using Tensorflow with H2O 

This notebook shows how to use the tensorflow backend to tackle a simple image classification problem.

We start by connecting to our h2o cluster:

In [ ]:
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
import os.path
from IPython.display import Image, display, HTML
import pandas as pd
import numpy as np
import random

PATH=os.path.expanduser("~/Dev/code/github/h2o-3")
h2o.init(port=54321, nthreads=-1)
if not H2ODeepWaterEstimator.available(): exit

In [ ]:
%matplotlib inline
from IPython.display import Image, display, HTML
import matplotlib.pyplot as plt

## Image Classification Task

H2O DeepWater allows you to specify a list of URIs (file paths) or URLs (links) to images, together with a response column (either a class membership (enum) or regression target (numeric)).

For this example, we use a small dataset that has a few hundred images, and three classes: cat, dog and mouse.

In [ ]:
frame = h2o.import_file(PATH + "/bigdata/laptop/deepwater/imagenet/cat_dog_mouse.csv")
print(frame.dim)
print(frame.head(5))

To build a LeNet image classification model in H2O, simply specify `network = "lenet"` and the **Tensorflow** backend to use the tensorflow lenet implementation:

In [ ]:
model = H2ODeepWaterEstimator(epochs=500, network = "lenet", backend="tensorflow")
model.train(x=[0],y=1, training_frame=frame)
model.show()

# DeepFeatures

We can also compute the output of any hidden layer, if we know its name.

In [ ]:
model.deepfeatures(frame, "fc1/Relu")

# Custom models

If you'd like to build your own Tensorflow network architecture, then this is easy as well.
In this example script, we are using the **Tensorflow** backend. 
Models can easily be imported/exported between H2O and Tensorflow since H2O uses Tensorflow's format for model definition.

In [ ]:
def simple_model(w, h, channels, classes):
    import json
    import tensorflow as tf
    from tensorflow.python.framework import ops
    # always create a new graph inside ipython or
    # the default one will be used and can lead to
    # unexpected behavior
    graph = tf.Graph() 
    with graph.as_default():
        size = w * h * channels
        x = tf.placeholder(tf.float32, [None, size])
        W = tf.Variable(tf.zeros([size, classes]))
        b = tf.Variable(tf.zeros([classes]))
        y = tf.matmul(x, W) + b

        predictions = tf.nn.softmax(y)
        
        # labels
        y_ = tf.placeholder(tf.float32, [None, classes])
        
        # train
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("predictions", predictions)
        # this is required by the h2o tensorflow backend
        global_step = tf.Variable(0, name="global_step", trainable=False)
        
        init = tf.initialize_all_variables()
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init.name)
        tf.add_to_collection("logits", y)
        saver = tf.train.Saver()
        meta = json.dumps({
                "inputs": {"batch_image_input": x.name, "categorical_labels": y_.name}, 
                "outputs": {"categorical_logits": y.name},
                "parameters": {"global_step": global_step.name},
        })
        print(meta)
        tf.add_to_collection("meta", meta)
        filename = "/tmp/lenet_tensorflow.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())
    return filename

In [ ]:
filename = simple_model(28, 28, 3, classes=3)

In [ ]:
model = H2ODeepWaterEstimator(epochs=500, 
                              network_definition_file=filename,  ## specify the model
                              image_shape=[28,28],  ## provide expected (or matching) image size
                              channels=3,
                              backend="tensorflow", 
                             ) 
model.train(x=[0], y=1, training_frame=frame)
model.show()

# Custom models with Keras

It is also possible to use libraries/APIs such as Keras to define the network architecture.

In [ ]:
import tensorflow as tf
import json
from keras.layers.core import Dense, Flatten, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
from keras.objectives import categorical_crossentropy
from tensorflow.python.framework import ops

def keras_model(w, h, channels, classes):
    # always create a new graph inside ipython or
    # the default one will be used and can lead to
    # unexpected behavior
    graph = tf.Graph() 
    with graph.as_default():
        size = w * h * channels
        # Input images fed via H2O
        inp = tf.placeholder(tf.float32, [None, size])
        # Actual labels used for training fed via H2O
        labels = tf.placeholder(tf.float32, [None, classes])

        # Keras network
        x = Reshape((w, h, channels))(inp)
        x = Conv2D(20, (5, 5), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)

        x = Conv2D(50, (5, 5), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)

        x = Flatten()(x)    

        x = Dense(500, activation='relu')(x)

        out = Dense(classes)(x)

        predictions = tf.nn.softmax(out)

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=out))
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

        init_op = tf.initialize_all_variables()

        # Metadata required by H2O
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init_op.name)
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("logits", out)
        tf.add_to_collection("predictions", predictions)

        meta = json.dumps({
                "inputs": {"batch_image_input": inp.name,
                           "categorical_labels": labels.name},
                "outputs": {"categorical_logits": out.name,
                            "layers": ','.join([m.name for m in tf.get_default_graph().get_operations()])},
                "parameters": {}
            })
        tf.add_to_collection("meta", meta)

        # Save the meta file with the graph
        saver = tf.train.Saver()
        filename = "/tmp/keras_tensorflow.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())

        return filename

In [ ]:
filename = keras_model(28, 28, 3, classes=3)

In [ ]:
model = H2ODeepWaterEstimator(epochs=50, 
                              network_definition_file=filename,  ## specify the model
                              image_shape=[28,28],  ## provide expected (or matching) image size
                              channels=3,
                              backend="tensorflow", 
                             ) 
model.train(x=[0], y=1, training_frame=frame)
model.show()